# Tunnel Transit Time Calculator
**Version: v0.1.0 (Alpha)**

## Overview
As stated in the [Transport Manifesto](https://github.com/dsposito/transport-manifesto), [tunnels](https://github.com/dsposito/transport-manifesto/tree/master/tunnels) are a mode of transport with a number of benefits. For example, tunnels can provide dramatically faster travel from point A to B due to 1.) the use of autonomous electric vehicles and 2.) support for higher speeds (155mph+).

**The purpose of this interactive notebook is to:**
1. Calculate the estimated time savings of traveling by tunnels vs traditional roadways
2. Easily inspect, edit and evolve the underlying logic that drives the comparison calculations \[[**view source code**](https://github.com/dsposito/transport-manifesto/blob/master/tunnels/transit-time-calculator/Tunnel%20Transit%20Time%20Calculator%20-%20Verbose.ipynb)]

In [1]:
import math
import os

from dotenv import load_dotenv, find_dotenv
import gmaps as gmaps_ui
import googlemaps as gmaps_api
import ipywidgets as widgets
from IPython.display import display, Markdown as md

In [2]:
load_dotenv(find_dotenv())

gmaps_ui.configure(api_key=os.getenv("GMAPS_API_KEY"))
gmaps_api = gmaps_api.Client(key=os.getenv("GMAPS_API_KEY"))

In [3]:
def handle_form_submit(element):
    """Handles the click even for the submit button.
    """
    
    # Clear any existing output (e.g., from previous form submit).
    form_output.clear_output()
    
    with form_output:
        # Validate the form inputs.
        if not input_origin.value or not input_destination.value:
            print("Please enter a valid address and try again.")
            return

        # Geocode the string addresses to coordinates (necessary for the gmaps widget).
        origin = geocode_address(input_origin.value)
        destination = geocode_address(input_destination.value)

        if not origin or not destination:
            print("Please enter a valid address and try again.")
            return

        # Calculate road comparison metrics (e.g., distance and time) by creating driving directions via Google Maps API.
        metrics_road = calculate_road_metrics(origin, destination)
        if not metrics_road:
            print("Unable to calculate road metrics.")
            return

        # Calculate tunnel comparison metrics using the total distance queried from Google Maps API.
        metrics_tunnel = calculate_tunnel_metrics(metrics_road["distance"]["value"])

        # Display the route in a Google Map and a table of comparison metrics.
        display_map(origin, destination)
        display_metrics(metrics_road, metrics_tunnel)

# Create the form fields.
input_origin = widgets.Text(description="Origin: ", value="Santa Monica, CA", layout={'width': '50%'})
input_destination = widgets.Text(description="Destination: ", value="Downtown Los Angeles", layout={'width': '50%'})
button_submit = widgets.Button(description="Calculate Route", button_style="Success")
button_submit.on_click(handle_form_submit)
form_output = widgets.Output()

# Display the form.
display(md("### Transit Time Comparison Calculator"))
display(input_origin, input_destination, button_submit, form_output)

### Transit Time Comparison Calculator

Text(value='Santa Monica, CA', description='Origin: ', layout=Layout(width='50%'))

Text(value='Downtown Los Angeles', description='Destination: ', layout=Layout(width='50%'))

Button(button_style='success', description='Calculate Route', style=ButtonStyle())

Output()

> **Note:** Checkout out the [**full source code notebook**](https://github.com/dsposito/transport-manifesto/blob/master/tunnels/transit-time-calculator/Tunnel%20Transit%20Time%20Calculator%20-%20Verbose.ipynb) for a walkthrough of how the comparison metrics are calculated.

In [4]:
def geocode_address(address: str) -> tuple:
    """Geocodes a string address into latitude/longitude coordinates.
    """
    
    response = gmaps_api.geocode(address)
    if not response:
        print("Unable to geocode address.")
        return

    location = response[0]["geometry"]["location"]

    return location["lat"], location["lng"]

In [5]:
def calculate_road_metrics(origin: tuple, destination: tuple) -> dict:
    """Calculates metrics for traveling a route via traditional roadways.
    
    The Google Maps API is queried to determine the total distance from origin to destination.
    """
    
    METERS_PER_FOOT = 0.3048
    FEET_PER_MILE = 5280
    
    CITY_TOP_SPEED = 40
    HIGHWAY_TOP_SPEED = 65
    
    # Query Google Maps API for directions data - which includes total distance and time.
    directions = gmaps_api.directions(origin, destination)
    if not directions:
        print("Unable to retrieve directions.")
        return
    
    # Convert distance from meters to miles.
    distance_miles = int(directions[0]["legs"][0]["distance"]["value"] / METERS_PER_FOOT / FEET_PER_MILE)
    
    # Convert time from seconds to minutes.
    time_minutes = directions[0]["legs"][0]["duration"]["value"] / 60
    
    # Calculate top speed by checking whether highway is used for any steps of the route.
    speed_max = CITY_TOP_SPEED
    for steps in directions[0]["legs"][0]["steps"]:
        if "ramp" in steps.get("maneuver", ""):
            speed_max = HIGHWAY_TOP_SPEED
            break
    
    # Calculate average speed: d = r*t
    time_hours = time_minutes / 60
    speed_avg = distance_miles / time_hours
    
    return {
        "distance": {
            "unit": "mi",
            "value": round(distance_miles, 1),
            "raw": distance_miles
        },
        "time": {
            "unit": "min",
            "value": round(time_minutes),
            "raw": time_minutes
        },
        "speed_max": {
            "unit": "mph",
            "value": round(speed_max),
            "raw": speed_max
        },
        "speed_avg": {
            "unit": "mph",
            "value": round(speed_avg),
            "raw": speed_avg
        }
    }

In [6]:
def simulate_tunnel_vehicle_kinematics(distance_trip: float) -> dict:
    """Simulates the acceleration of an AEV in a tunnel to determine the top speed and total time of travel.
    """
    
    # Assumptions for Tesla Model X vehicle.
    VEHICLE_TOP_SPEED = 155 # mph
    VEHICLE_TOP_SPEED_TIME = 45 # seconds (~0.157 g-force: http://www.procato.com/convert)

    # Assume constant rate of acceleration/deceleration.
    VEHICLE_DELTA_SPEED_TIME = VEHICLE_TOP_SPEED / VEHICLE_TOP_SPEED_TIME

    speed_current = 0
    for time_elapsed in range(0, VEHICLE_TOP_SPEED_TIME + 1):
        # Calculate distance traveled up to this point using the kinematic displacement formula: d = 0.5 * v * t
        velocity_current = speed_current / (60 * 60)
        distance_current = 0.5 * velocity_current * time_elapsed

        velocity_next = (speed_current + VEHICLE_DELTA_SPEED_TIME) / (60 * 60)
        distance_next = 0.5 * velocity_next * (time_elapsed + 1)

        # Cap the velocity if the upcoming distance traveled would be more than half
        # of the total trip distance (outcome would be to begin decelerating).
        if distance_next * 2 >= distance_trip:
            distance_current = distance_trip / 2
            velocity_current = distance_current / (60 * 60)

            break

        # Ensure current speed never exceeds top speed.
        speed_current = min(speed_current + VEHICLE_DELTA_SPEED_TIME, VEHICLE_TOP_SPEED)
        
    return {
        "distance": distance_current,
        "time": time_elapsed,
        "mph_max": speed_current,
        "velocity_max": velocity_current
    }

In [7]:
def calculate_tunnel_metrics(distance_trip: float) -> dict:
    """Calculates metrics for traveling a route via tunnels.
    """
    
    acceleration = simulate_tunnel_vehicle_kinematics(distance_trip)
    
    # Miles
    distance_accelerating = acceleration["distance"]
    distance_decelerating = acceleration["distance"]
    distance_cruising = distance_trip - distance_accelerating - distance_decelerating

    # Seconds
    time_accelerating = acceleration["time"]
    time_decelerating = acceleration["time"]
    time_cruising = distance_cruising / acceleration["velocity_max"]
    time_total = time_accelerating + time_cruising + time_decelerating

    time_total_minutes = time_total / 60
    time_total_hours = time_total_minutes / 60

    speed_max = acceleration["mph_max"]
    speed_avg = distance_trip / time_total_hours

    return {
        "distance": {
            "unit": "mi",
            "value": round(distance_trip, 1),
            "raw": distance_trip
        },
        "time": {
            "unit": "min",
            "value": math.ceil(time_total_minutes), # Round up for a more conservative estimate.
            "raw": time_total_minutes
        },
        "speed_max": {
            "unit": "mph",
            "value": round(speed_max),
            "raw": speed_max
        },
        "speed_avg": {
            "unit": "mph",
            "value": round(speed_avg),
            "raw": speed_avg
        }
    }

In [8]:
def display_map(origin: tuple, destination: tuple):
    route = gmaps_ui.directions_layer(origin, destination)

    map = gmaps_ui.figure()
    map.add_layer(route)
    display(map)

def display_metrics(road: dict, tunnel: dict):
    comparison = f"""| Metric | Road | Tunnel |
    | --- | --- | --- |
    | Distance | {road['distance']['value']} {road['distance']['unit']} | {tunnel['distance']['value']} {tunnel['distance']['unit']} |
    | Top Speed | {road['speed_max']['value']} {road['speed_max']['unit']} | {tunnel['speed_max']['value']} {tunnel['speed_max']['unit']} |
    | Avg Speed | {road['speed_avg']['value']} {road['speed_avg']['unit']} | {tunnel['speed_avg']['value']} {tunnel['speed_avg']['unit']} |
    | **Transit Time** | **{road['time']['value']} {road['time']['unit']}** | **{tunnel['time']['value']} {tunnel['time']['unit']}** |"""

    time_saved = road['time']['value'] - tunnel['time']['value']
    unit = "minutes"
    if time_saved > 60:
        time_saved /= 60
        unit = "hours"
    
    # Render the comparison metrics as a markdown table.
    display(md("### Traditional Roads vs Tunnels Comparison"))   
    display(md(f"**Total Time Saved via Tunnels:** {time_saved} {unit}"))
    display(md("<br>"))
    display(md(comparison))